# AUTOGRAD: AUTOMATIC DIFFERENTIATION
PyTorch 的中心包是 `autograd`包。

`autograd` 包提供了，Tensors上所有操作的自动分化。这是一个 define-by-run 的框架，这意味着您的反向传播代码是由代码运行方式决定，所以可能每次迭代都不同。

我们可以通过几个例子来看

## Tensor

`torch.Tensor` 是这个包的核心 class. 如果我们将它的属性(attribute) `.requires_grad` 设置为 `True`,它将开始记录所有运算。当你完成计算时你可以用 `.backward()` 将所有梯度的渐变自动计算出来，这个 Tensor 的梯度也会被累积到 `.grad` 属性里。

要停止 Tensor 追踪历史记录，可以用 `.detach()`将其从计算历史记录里剥离出来，可以避免后续的计算被记录。

为了防止追踪历史记录（以及使用内存），你也可以将代码块用 `torch.no_grad()`包装起来。这在评估模型的时候尤其有用，这是因为模型可能具有可训练的参数，并且 `requires_grad=True`，但是我们不需要这些梯度。

另外还有一个 class 对于自动分级来说非常重要，它是 `Function`.

`Tensor` 和 `Function`互相连接，形成一个无环图，该图对完整的计算历史进行编码。每一个 Tensor 都有一个 `.grad_fn` 的属性，指向一个 `Function`，这个 `Function` 是创建这个 `Tensor`的（除了 有部分用户 创建的 Tensors 它们的 `grad_fn` 是 None）

如果你想计算导数(derivatives)，你可以使用在Tensor上使用 `.backward()`。如果 `Tensor`是一个标量(scalar, 代表其是一个单元素数据)，您无需为 `backward()` 指定任何参数，然而如果它有更多的元素，则我们需要指定一个 `gradient` 参数，来匹配 Tensor的形状。



In [1]:
import torch

创建一个 Tensor 并且设置 `require_grad=True` 来跟踪计算

In [2]:
x = torch.ones(2,2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


做一个 tensor 计算

In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


`y`是被一个运算创建的，所以它有一个 `grad_fn

In [4]:
print(y.grad_fn)

在 `y` 上做更多的计算

In [5]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


`.requires_grad_(...)` 改变了一个已存在 Tensor 的 `requires_grad` 的值.默认情况下这个值是 `Flase`

In [9]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
None
True


## Gradients 梯度
现在开始反向传播。由于输出包含一个单标量，所以 `out.backward()`